# Marketing Mix Modeling (MMM) Training Pipeline

This notebook implements a simplified Marketing Mix Model using Ridge Regression to attribute revenue to marketing channels and estimate ROI.

**Objectives:**
1. Load weekly spend and revenue data.
2. Apply Adstock transformations (lag/decay effect).
3. Train a regression model.
4. Calculate ROI and Response Curves.
5. Save results to Snowflake for the Streamlit app.

In [ ]:
import pandas as pd
import numpy as np
from snowflake.snowpark.context import get_active_session
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
session = get_active_session()

In [ ]:
# Load view from MMM schema
df_input = session.table("MMM.V_MMM_INPUT_WEEKLY").to_pandas()
print(f"Loaded {len(df_input)} rows.")
df_input.head()

In [ ]:
def geometric_adstock(x, decay=0.5):
    """Apply geometric adstock transformation."""
    x_decayed = np.zeros_like(x)
    if len(x) > 0:
        x_decayed[0] = x[0]
        for i in range(1, len(x)):
            x_decayed[i] = x[i] + decay * x_decayed[i-1]
    return x_decayed

In [ ]:
# Pivot to wide format for modeling: Week x Channel
df_pivot = df_input.pivot_table(
    index='WEEK_START', 
    columns='CHANNEL', 
    values='SPEND', 
    aggfunc='sum'
).fillna(0).sort_index()

# Get revenue target (assuming single target for simplicity)
df_revenue = df_input.groupby('WEEK_START')['REVENUE'].sum().sort_index()

# Align indices
common_idx = df_pivot.index.intersection(df_revenue.index)
X_raw = df_pivot.loc[common_idx]
y = df_revenue.loc[common_idx]

# Apply Adstock to each channel (simplified fixed decay for demo)
X_adstock = X_raw.copy()
for col in X_adstock.columns:
    X_adstock[col] = geometric_adstock(X_adstock[col].values, decay=0.6)

# Add macro signals if available
# (Skipping for simple Ridge demo to ensure stability, but would add columns here)

print("Feature engineering complete.")

In [ ]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_adstock)

# Train Ridge Regression (L2 regularization to handle collinearity)
model = Ridge(alpha=1.0, positive=True) # positive=True enforces positive coefficients (no negative ROI)
model.fit(X_scaled, y)

r2 = model.score(X_scaled, y)
print(f"Model Trained. R2 Score: {r2:.4f}")

In [ ]:
# Decompose attribution
coeffs = dict(zip(X_adstock.columns, model.coef_ / scaler.scale_)) # Adjust coefs back to original scale
intercept = model.intercept_ - np.sum(model.coef_ * scaler.mean_ / scaler.scale_)

results = []
total_spend = X_raw.sum()

for channel in X_raw.columns:
    coef = coeffs[channel]
    spend = total_spend[channel]
    # Simple attribution: Total Adstock Volume * Coef
    # Note: This is an approximation. True attribution sums contribution per week.
    total_adstock = X_adstock[channel].sum()
    attributed_rev = total_adstock * coef
    
    roi = attributed_rev / spend if spend > 0 else 0
    marginal_roi = coef # Linear model, marginal ROI is constant (slope)
    
    results.append({
        'MODEL_VERSION': 'v1.0',
        'CHANNEL': channel,
        'COEFF_WEIGHT': float(coef),
        'ROI': float(roi),
        'MARGINAL_ROI': float(marginal_roi),
        'OPTIMAL_SPEND': float(spend * 1.1) # Placeholder optimization
    })

df_results = pd.DataFrame(results)
df_results

In [ ]:
# Generate simulation curves for "What-If" tool
curves = []
simulation_points = np.linspace(0, 2.0, 50) # 0% to 200% of average spend

avg_spend = X_raw.mean()

for channel in X_raw.columns:
    base_spend = avg_spend[channel]
    coef = coeffs[channel]
    
    for mult in simulation_points:
        sim_spend = base_spend * mult
        # Linear response for Ridge, but usually Diminishing Returns (Hill function)
        # We'll simulate a saturation effect for the curve visualization manually here
        # Revenue = Coef * Spend ^ 0.8 (Simple Power Law for saturation demo)
        sim_rev = coef * (sim_spend ** 0.8 * base_spend ** 0.2) 
        
        curves.append({
            'MODEL_VERSION': 'v1.0',
            'CHANNEL': channel,
            'SPEND': float(sim_spend),
            'PREDICTED_REVENUE': float(sim_rev)
        })

df_curves = pd.DataFrame(curves)

In [ ]:
# Save to MMM schema
session.create_dataframe(df_results).write.mode("overwrite").save_as_table("MMM.MODEL_RESULTS")
session.create_dataframe(df_curves).write.mode("overwrite").save_as_table("MMM.RESPONSE_CURVES")
print("Results saved to Snowflake tables.")